In [1]:
import torch
print(torch.__version__)       # should now be something like 2.8.0+cu121
print(torch.version.cuda)      # should say 12.1
print("CUDA available:", torch.cuda.is_available())  # ✅ should be True
print("GPU count:", torch.cuda.device_count())       # should be > 0
print("GPU name:", torch.cuda.get_device_name(0))    # should say "NVIDIA GeForce RTX 3060"


2.5.1+cu121
12.1
CUDA available: True
GPU count: 1
GPU name: NVIDIA GeForce RTX 4060 Laptop GPU


In [ ]:
%matplotlib inline
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from ultralytics import YOLO

# --- ✅ Config ---
model_path = r"C:\Users\vkr30\Image Segmentation_Plant Disease\Yolov11 Variants for PDP\best.pt"
data_dir = r"C:\Users\vkr30\Image Segmentation_Plant Disease\PlantVillageY1500\train"
img_size = 224
batch_size = 16
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
save_path = r"C:\Users\vkr30\Image Segmentation_Plant Disease\plant_disease_cls\Top5_Confusion_Matrix.png"

# --- ✅ Transform ---
transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
])

# --- ✅ Dataset & DataLoader ---
val_dataset = ImageFolder(root=data_dir, transform=transform)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
class_names = val_dataset.classes
num_classes = len(class_names)

# --- ✅ Load YOLOv11 classification model ---
model = YOLO(model_path)
model.to(device).eval()

# --- ✅ Inference loop for Top-5 predictions ---
top5_preds, true_labels = [], []

with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        labels = labels.to(device)

        results = model(images, verbose=False)

        for result, label in zip(results, labels):
            probs = result.probs.data  # shape: [num_classes]
            top5 = torch.topk(probs, k=5).indices.cpu().numpy()
            top5_preds.append(top5)
            true_labels.append(label.item())

# --- ✅ Generate Top-5 Confusion Matrix (Raw Counts) ---
confusion_top5 = np.zeros((num_classes, num_classes), dtype=int)
for true, top5 in zip(true_labels, top5_preds):
    for pred in top5:
        confusion_top5[true, pred] += 1

# --- ✅ Plot and save confusion matrix ---
plt.figure(figsize=(14, 12))
sns.heatmap(confusion_top5, annot=True, fmt='d', cmap="Blues",
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel("Predicted Label (Top-5 hits)")
plt.ylabel("True Label")
plt.title("Top-5 Confusion Matrix (Raw Counts)")
plt.xticks(rotation=45, ha="right")
plt.yticks(rotation=0)
plt.tight_layout()

plt.savefig(save_path, dpi=300)
plt.show()

print(f"✅ Confusion matrix saved at: {save_path}")
